In [1]:
from langchain.memory.buffer import ConversationBufferMemory
from langchain.chains.conversation.base import ConversationChain


In [2]:
import os
from dotenv import load_dotenv

load_dotenv()
print("Done")

Done


In [3]:
%run ../app/config.py

In [4]:
import os
import requests
import json
import time

import jwt
import numpy as np
from langchain_community.llms import YandexGPT
from langchain_community.embeddings.yandex import YandexGPTEmbeddings


now = int(time.time())
payload = {
        'aud': API_URL,
        'iss': SERVICE_ACCOUNT_ID,
        'iat': now,
        'exp': now + 360}

# Формирование JWT
encoded_token = jwt.encode(
    payload,
    PRIVATE_KEY,
    algorithm='PS256',
    headers={'kid': KEY_ID})

x = requests.post(API_URL,  headers={'Content-Type': 'application/json'}, json = {'jwt': encoded_token}).json()
os.environ['YC_IAM_TOKEN'] = x['iamToken']
llm =  YandexGPT(model_uri=GPT_MODEL_URL)
embeddings = YandexGPTEmbeddings(model_uri=EMB_MODEL_URL, folder_id=FOLDER_ID)

In [5]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You're an assistant who's good at styling. Respond in 20 words or fewer",
        ),
        MessagesPlaceholder(variable_name="history"),
        ("human", "{input}"),
    ]
)
runnable = prompt | llm

In [6]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

store = {}


def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]


with_message_history = RunnableWithMessageHistory(
    runnable,
    get_session_history,
    input_messages_key="input",
    history_messages_key="history",
)

In [7]:
with_message_history.invoke(
    {"input": "What did I ask?"},
    config={"configurable": {"session_id": "abc123"}},
)

'System: What can I help you with?'

In [9]:
store['abc123']

InMemoryChatMessageHistory(messages=[HumanMessage(content='What did I ask?'), AIMessage(content='System: What can I help you with?')])

In [34]:
with_message_history.get_session_history("abc123")

InMemoryChatMessageHistory(messages=[HumanMessage(content='What does cosine mean?'), AIMessage(content='Cosine is a trigonometric function.'), HumanMessage(content='What did I ask?'), AIMessage(content='System: Styling is not my expertise, but I can help you with that. What would you like to know about styling?')])

In [22]:
from langchain_core.chat_history import InMemoryChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory
# from langchain_openai import ChatOpenAI


store = {}  # memory is maintained outside the chain

def get_session_history(session_id: str) -> InMemoryChatMessageHistory:
    if session_id not in store:
        store[session_id] = InMemoryChatMessageHistory()
    return store[session_id]

# llm = ChatOpenAI(model="gpt-3.5-turbo-0125")

chain = RunnableWithMessageHistory(llm, get_session_history)


In [25]:
chain.invoke(
    "Hi I'm Bob.",
    config={"configurable": {"session_id": "1"}},
)  # session_id determines thread

Error in RootListenersTracer.on_llm_end callback: KeyError('message')


'AI: Hello, Bob! How are you?'

In [26]:
chain.invoke(
    "I'm fine. What's my name?",
    config={"configurable": {"session_id": "1"}},
)  # session_id determines thread

Error in RootListenersTracer.on_llm_end callback: KeyError('message')


'AI: Can you please specify your full name?'